## качаем данные

In [2]:
!pip install bs4

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=a5fb47b782c1281189b2ccaf604495f3a8380a270bf7abb4dd41c02351af0e4f
  Stored in directory: /Users/rk/Library/Caches/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4

[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [3]:
import requests
from bs4 import BeautifulSoup 
import time
from urllib.parse import urlparse, urljoin
from tqdm.notebook import tqdm

In [5]:
## найти id каждого вопроса на страницах
page = 1
main_url = 'https://zpp.rospotrebnadzor.ru/Forum/Appeals/AjaxindexList?page={page}&searchtext=&categories=[]'

def ids_from_page(soup):
    return [i.get('href') for i  in soup.findAll('a', {'class':'appeal-title-link'})]
    
urls = []
for page in tqdm(range(1, 600), desc='Pages'):
    time.sleep(0.5)
    res = requests.get(main_url.format(page=page))  
    soup = BeautifulSoup(res.text)
    urls.extend(ids_from_page(soup))

Pages:   0%|          | 0/599 [00:00<?, ?it/s]

In [6]:
## имея id запросов достанем тексты на страницах
domain = 'https://zpp.rospotrebnadzor.ru'

def get_text_page_question(soup):
    return soup.findAll('p', {'class':'appeal-details-message'})[0].text
def get_text_page_answer(soup):
    return soup.findAll('p', {'class':'appeal-comments-message'})[0].text

question = []
answers = []

for url in tqdm(urls, desc='Pages'):
    time.sleep(0.5)
    res = requests.get(urljoin(domain, url))  
    soup = BeautifulSoup(res.text)
    
    question.append(get_text_page_question(soup))
    try:
        answers.append(get_text_page_answer(soup))
    except:
        answers.append('No answer')

Pages:   0%|          | 0/2995 [00:00<?, ?it/s]

In [7]:
## посмотрим, что скачалось
N = 2
print('Q:', question[N])
print()
print('A:', answers[N])

Q: Я врач-инфекционист хочу открыть инфекционный кабинет в нежилом здании. Подскажите какие требования необходимо соблюсти помимо отдельного входа и туалета.

A: Здравствуйте, Елена! При осуществлении медицинской деятельности и открытии инфекционного кабинета необходимо соблюдать следующие требования санитарного законодательства: 
- СанПиН 3.3686-21 "Санитарно-эпидемиологические требования по профилактике инфекционных болезней", утвержденных постановлением Главного государственного санитарного врача РФ от 28.01.2021 N 4;
 - СП 2.1.3678-20 "Санитарно-эпидемиологические требования к эксплуатации помещений, зданий, сооружений, оборудования и транспорта, а также условиям деятельности хозяйствующих субъектов, осуществляющих продажу товаров, выполнение работ или оказание услуг", утвержденных постановлением Главного государственного санитарного врача РФ от 24.12.2020 N 44 . 




In [11]:
import pandas as pd
df = pd.DataFrame(data={'questions': question, 'answers': answers})
df.head()

,questions,answers
0,Помогите пожалуйста в расторжении договора с П...,Уважаемый Игорь!\r\nВ соответствии с пунктом 4...
1,"Жалоба на детскую поликлинику номер 15, по адр...","Ксения!\r\n Вопросы, поставленные в Ва..."
2,Я врач-инфекционист хочу открыть инфекционный ...,"Здравствуйте, Елена! При осуществлении медицин..."
3,"Возле подъезда лежат мертвые птицы, УК Наш Гор...","Ольга,добрый день! В Московской области с таки..."
4,Прошу провести проверку по факту размещения ра...,"Вячеслав,добрый день! Вам необходимо перенапра..."


In [13]:
df.tail()

,questions,answers
2990,Добрый день.\r\nПрошу разъяснить - относятся л...,"Денис Валерьевич, здравствуйте. Да, предприяти..."
2991,Мной был куплен товар ( робот-пылесос) у ОБЩЕС...,Добрый день! Вами не указано каким путем приоб...
2992,Вот такое объявление установлено в кофейне Cof...,"Уважаемый Потребитель ! Информация, размещен..."
2993,15.11.2021 и 16.11.2021 года совершала покупку...,Уважаемая Анна!\r\n\r\n В целях объект...
2994,Добрый день! Есть ли ограничения въезда группы...,Уважаемая Ирина! Ограничения на въезд организо...


In [12]:
df.to_csv('rospotrebnadzor.csv')